In [1]:
import duckdb
import pandas as pd
from datetime import datetime, timedelta
import sys

def analyze_timestamp(timestamp_value):
    """Analyze a timestamp value and print various possible interpretations"""
    print(f"\nAnalyzing timestamp: {timestamp_value}")
    print("-" * 50)
    
    # Standard epochs
    epochs = {
        "Windows (1601-01-01)": datetime(1601, 1, 1),
        "Unix (1970-01-01)": datetime(1970, 1, 1),
        "NTP (1900-01-01)": datetime(1900, 1, 1),
        "Mac (1904-01-01)": datetime(1904, 1, 1),
    }
    
    # Scaling factors to try
    scaling_factors = {
        "Nanoseconds ÷ 100": lambda ts, epoch: epoch + timedelta(microseconds=ts/10),
        "Microseconds": lambda ts, epoch: epoch + timedelta(microseconds=ts),
        "Milliseconds": lambda ts, epoch: epoch + timedelta(milliseconds=ts),
        "Seconds": lambda ts, epoch: epoch + timedelta(seconds=ts),
        "Minutes": lambda ts, epoch: epoch + timedelta(minutes=ts),
        "Hours": lambda ts, epoch: epoch + timedelta(hours=ts),
        "Days": lambda ts, epoch: epoch + timedelta(days=ts),
    }
    
    # Custom scaling factors (for common browser/system implementations)
    custom_conversions = {
        "Chrome/WebKit (μs since 1601, special scale)": lambda ts: epochs["Windows (1601-01-01)"] + timedelta(microseconds=ts/1000),
        "Windows Performance Counter": lambda ts: epochs["Windows (1601-01-01)"] + timedelta(seconds=ts/10_000_000),
        "Windows High Resolution": lambda ts: epochs["Windows (1601-01-01)"] + timedelta(seconds=ts/1_000_000),
        "Unix Nanoseconds": lambda ts: epochs["Unix (1970-01-01)"] + timedelta(microseconds=ts/1000),
        "Unix High Precision": lambda ts: epochs["Unix (1970-01-01)"] + timedelta(seconds=ts/1_000_000),
    }
    
    # Try standard combinations
    results = []
    
    for epoch_name, epoch in epochs.items():
        for scaling_name, scaling_func in scaling_factors.items():
            try:
                date = scaling_func(timestamp_value, epoch)
                results.append({
                    "interpretation": f"{epoch_name} + {scaling_name}",
                    "date": date,
                    "year": date.year
                })
            except (OverflowError, ValueError) as e:
                pass  # Skip invalid combinations
    
    # Try custom conversions
    for custom_name, custom_func in custom_conversions.items():
        try:
            date = custom_func(timestamp_value)
            results.append({
                "interpretation": custom_name,
                "date": date,
                "year": date.year
            })
        except (OverflowError, ValueError) as e:
            pass
    
    # Sort results by year (recent dates first)
    results.sort(key=lambda x: x["year"], reverse=True)
    
    # Print all interpretations
    current_year = datetime.now().year
    
    print(f"{'Interpretation':<40} | {'Converted Date':<30} | {'Likelihood':<10}")
    print("-" * 85)
    
    for result in results:
        date = result["date"]
        year = date.year
        
        # Determine likelihood
        likelihood = "Low"
        if 2000 <= year <= current_year:
            likelihood = "High"
        elif 1990 <= year <= 2030:
            likelihood = "Medium"
        
        print(f"{result['interpretation']:<40} | {date.strftime('%Y-%m-%d %H:%M:%S.%f'):<30} | {likelihood:<10}")



timestamp_value=13346527881026989    
analyze_timestamp(timestamp_value)


Analyzing timestamp: 13346527881026989
--------------------------------------------------
Interpretation                           | Converted Date                 | Likelihood
-------------------------------------------------------------------------------------
Unix (1970-01-01) + Microseconds         | 2392-12-07 16:51:21.026989     | Low       
Unix High Precision                      | 2392-12-07 16:51:21.026989     | Low       
Mac (1904-01-01) + Microseconds          | 2326-12-07 16:51:21.026989     | Low       
NTP (1900-01-01) + Microseconds          | 2322-12-08 16:51:21.026989     | Low       
Windows (1601-01-01) + Microseconds      | 2023-12-08 16:51:21.026989     | High      
Windows High Resolution                  | 2023-12-08 16:51:21.026989     | High      
Unix (1970-01-01) + Nanoseconds ÷ 100    | 2012-04-17 08:53:08.102699     | High      
Unix Nanoseconds                         | 1970-06-04 11:22:07.881027     | Low       
Mac (1904-01-01) + Nanoseconds ÷ 100    

In [3]:
import duckdb

# First, let's test a direct conversion without creating a function
query = """
SELECT STRFTIME(
    TIMESTAMP '1601-01-01 00:00:00' + 
    (13346527881026989 / 1000000) * INTERVAL '1 second' +
    (13346527881026989 % 1000000) * INTERVAL '1 microsecond',
    '%d%m%Y'
) AS formatted_date;
"""

try:
    result = duckdb.sql(query).fetchone()[0]
    print(f"Formatted date: {result}")
except Exception as e:
    print(f"Error: {e}")

Error: Binder Error: No function matches the given name and argument types '*(DOUBLE, INTERVAL)'. You might need to add explicit type casts.
	Candidate functions:
	*(TINYINT, TINYINT) -> TINYINT
	*(SMALLINT, SMALLINT) -> SMALLINT
	*(INTEGER, INTEGER) -> INTEGER
	*(BIGINT, BIGINT) -> BIGINT
	*(HUGEINT, HUGEINT) -> HUGEINT
	*(FLOAT, FLOAT) -> FLOAT
	*(DOUBLE, DOUBLE) -> DOUBLE
	*(DECIMAL, DECIMAL) -> DECIMAL
	*(UTINYINT, UTINYINT) -> UTINYINT
	*(USMALLINT, USMALLINT) -> USMALLINT
	*(UINTEGER, UINTEGER) -> UINTEGER
	*(UBIGINT, UBIGINT) -> UBIGINT
	*(UHUGEINT, UHUGEINT) -> UHUGEINT
	*(INTERVAL, BIGINT) -> INTERVAL
	*(BIGINT, INTERVAL) -> INTERVAL



In [7]:
import duckdb
from datetime import datetime, timedelta

# Register a Python function with DuckDB
def windows_timestamp_to_ddmmyyyy(timestamp_value):
    # Windows epoch is January 1, 1601
    windows_epoch = datetime(1601, 1, 1)
    
    # Convert timestamp to datetime (microseconds since Windows epoch)
    dt = windows_epoch + timedelta(microseconds=timestamp_value)
    
    # Format as ddmmyyyy
    return dt.strftime("%d%m%Y")

# Register the function with DuckDB
duckdb.create_function('win_ts_to_ddmmyyyy', windows_timestamp_to_ddmmyyyy, ['DOUBLE'], 'VARCHAR')

# Test the function
test_query = """
SELECT win_ts_to_ddmmyyyy(13346527881026989::DOUBLE) AS formatted_date;
"""

result = duckdb.sql(test_query).fetchone()[0]
print(f"Formatted date: {result}")

# Example with a table
table_query = """
-- Create a sample table with timestamp values
CREATE OR REPLACE TABLE sample_timestamps AS
SELECT 13346527881026989::DOUBLE AS ts_value
UNION ALL SELECT 13346614281026989::DOUBLE; -- One day later

-- Apply the function to the table
SELECT ts_value, win_ts_to_ddmmyyyy(ts_value) AS formatted_date
FROM sample_timestamps;
"""

print("\nSample table results:")
print(duckdb.sql(table_query).fetchdf())

Formatted date: 08122023

Sample table results:
       ts_value formatted_date
0  1.334653e+16       08122023
1  1.334661e+16       09122023


C:\Users\hp\AppData\Local\Temp\ipykernel_15396\2326181165.py:16: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  duckdb.create_function('win_ts_to_ddmmyyyy', windows_timestamp_to_ddmmyyyy, ['DOUBLE'], 'VARCHAR')


In [ ]:
from datetime import datetime, timedelta

def windows_timestamp_to_ddmmyyyy(timestamp_value):
    # Windows epoch is January 1, 1601
    windows_epoch = datetime(1601, 1, 1)
    
    # Convert timestamp to datetime (microseconds since Windows epoch)
    dt = windows_epoch + timedelta(microseconds=timestamp_value)
    
    # Format as ddmmyyyy
    return dt.strftime("%d%m%Y")

# Test with your timestamp
timestamp_value = 13346527881026989
formatted_date = windows_timestamp_to_ddmmyyyy(timestamp_value)
print(f"Formatted date: {formatted_date}")

Formatted date: 08122023


In [13]:
d_sql_query = """
-- DuckDB pure SQL function to convert Windows timestamp to date in ddmmyyyy format
-- Windows timestamp is microseconds since 1601-01-01

-- Create the function
CREATE OR REPLACE FUNCTION win_ts_to_ddmmyyyy(ts DECIMAL(38,0)) RETURNS VARCHAR AS $$
    -- Use epoch_ms function which returns milliseconds since 1970-01-01
    -- We need to convert from Windows epoch (1601-01-01) to Unix epoch (1970-01-01)
    WITH decomposed AS (
        SELECT
            -- Break down the timestamp into manageable parts
            -- Windows to Unix epoch offset in microseconds (difference between 1601-01-01 and 1970-01-01)
            11644473600000000::DECIMAL(38,0) AS windows_to_unix_offset,
            -- Extract seconds from microseconds
            CAST(ts / 1000000 AS DECIMAL(38,0)) AS seconds_part,
            -- Extract microseconds remainder
            CAST(ts % 1000000 AS INTEGER) AS micros_part
    )
    SELECT STRFTIME(
        -- Calculate date by adding to 1601-01-01
        TIMESTAMP '1601-01-01 00:00:00' 
        + seconds_part * INTERVAL '1 second'
        + micros_part * INTERVAL '1 microsecond',
        '%d%m%Y'
    ) AS formatted_date
    FROM decomposed;
$$;

-- Test the function with our example timestamp
SELECT win_ts_to_ddmmyyyy(13346527881026989::DECIMAL(38,0)) AS formatted_date;

-- Example with a table
CREATE OR REPLACE TABLE sample_timestamps AS
SELECT 13346527881026989::DECIMAL(38,0) AS ts_value, 'Original date' AS description
UNION ALL SELECT 13346614281026989::DECIMAL(38,0), 'One day later'  -- One day later
UNION ALL SELECT 13346095881026989::DECIMAL(38,0), 'Five days earlier';  -- Five days earlier

-- Apply the function to the table
SELECT 
    ts_value, 
    description, 
    win_ts_to_ddmmyyyy(ts_value) AS formatted_date
FROM sample_timestamps
ORDER BY ts_value;
"""

print(duckdb.sql(d_sql_query).fetchdf())

ParserException: Parser Error: syntax error at or near "DECIMAL"

In [21]:
import duckdb

# Define the timestamp value
timestamp_value = 13346527881026989

# Here's a DuckDB query that avoids integer overflow
sql_query = f"""
-- Pre-calculate large constants to avoid overflow
WITH constants AS (
    SELECT 
        CAST(86400 AS BIGINT) AS seconds_per_day,
        CAST(1000000 AS BIGINT) AS micros_per_second,
        CAST(3600 AS BIGINT) AS seconds_per_hour,
        CAST(60 AS BIGINT) AS seconds_per_minute
),
-- Break down the timestamp into days and remaining time
time_parts AS (
    SELECT 
        -- Calculate days since Windows epoch (using BIGINT for all operations)
        CAST({timestamp_value} / (constants.micros_per_second * constants.seconds_per_day) AS INTEGER) AS days_since_epoch,
        
        -- Calculate remaining seconds in the day (ensure we use BIGINT)
        CAST(({timestamp_value} % (constants.micros_per_second * constants.seconds_per_day)) / constants.micros_per_second AS INTEGER) AS remaining_seconds,
        
        -- Calculate microseconds
        CAST({timestamp_value} % constants.micros_per_second AS INTEGER) AS remaining_microseconds
    FROM constants
),
-- Calculate the date
calculated_date AS (
    SELECT 
        -- Add days to Windows epoch
        DATE '1601-01-01' + days_since_epoch AS base_date,
        
        -- Create a time value for the remaining seconds and microseconds
        MAKE_TIME(
            CAST(remaining_seconds / 3600 AS BIGINT),           -- hours
            CAST((remaining_seconds % 3600) / 60 AS BIGINT),    -- minutes
            CAST((remaining_seconds % 60) AS DOUBLE) +          -- seconds
            (remaining_microseconds / 1000000.0)                -- fractional seconds
        ) AS remaining_time
    FROM time_parts
),
-- Combine date and time
full_timestamp AS (
    SELECT 
        base_date + remaining_time AS date_time
    FROM calculated_date
)
-- Format the final result
SELECT STRFTIME(date_time, '%d%m%Y') AS formatted_date
FROM full_timestamp;
"""

try:
    result = duckdb.sql(sql_query).fetchone()[0]
    print(f"Formatted date: {result}")
except Exception as e:
    print(f"Error: {e}")

Formatted date: 09122023
